<a href="https://colab.research.google.com/github/paulawtck/naocaia-imersao-ia-alura/blob/main/NaoCaIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação das bibliotecas necessárias
!pip -q install google-genai
!pip install -q google-adk

# Imports iniciais
import os
import warnings
import textwrap
from datetime import date
from google.colab import userdata
from IPython.display import display, Markdown, HTML
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types


warnings.filterwarnings("ignore")

# Configura a chave da API do Gemini
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except Exception as e:
    print("Erro ao recuperar a chave da API:", e)

# Cliente Gemini
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"


# Sessão e histórico globais para manter a conversa
session_service = InMemorySessionService()
session_ids = {}  # ID da sessão por agente
conversas = {}    # Histórico de mensagens por agente


# Função auxiliar para chamar um agente com nova sessão (sem histórico)
def call_agent(agent: Agent, message_text: str) -> str:
  try:
    # Cria sessão temporária só para essa mensagem, sem histórico
    session_service_tmp = InMemorySessionService()
    session_tmp = session_service_tmp.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service_tmp)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response
  except Exception as e:
    return f"❌ Erro ao executar o agente: {e}"


# Função para exibir texto formatado em markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


# Função para iniciar chat com histórico
def iniciar_chat_continuo(nome_agente, descricao, instrucao):
    print(f"\n💬 Você está no {descricao}.")
    print("🤖 Digite sua dúvida ou 0 para voltar ao menu.\n")


    # Cria agente
    agent = Agent(
        name=nome_agente,
        model=MODEL_ID,
        instruction=instrucao,
        description=descricao
    )


    # Cria ou recupera sessão e histórico
    if nome_agente not in session_ids:
        session_id = f"{nome_agente}_sessao"
        session_ids[nome_agente] = session_id
        session_service.create_session(app_name=agent.name, user_id="user1", session_id=session_id)


    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    while True:
      try:
        pergunta = input("🔍 Pergunta: ")
        if pergunta.strip() == "0":
            break


        content_user = types.Content(role="user", parts=[types.Part(text=pergunta)])


        resposta_final = ""
        for event in runner.run(
            user_id="user1",
            session_id=session_ids[nome_agente],
            new_message=content_user
        ):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text:
                        resposta_final += part.text + "\n"

        display(to_markdown(resposta_final))

      except Exception as e:
          print(f"\n❌ Erro ao conversar com o agente: {e}")


# Funções de agentes para respostas isoladas (sem chat)
def agente_verificador_link(link):
    agent = Agent(
        name="agente_verificador_link",
        model=MODEL_ID,
        instruction="""
         Você é um especialista em segurança digital, altamente criterioso e detalhista.
        Sua tarefa é analisar links suspeitos ou desconhecidos.

        1. Avalie a taxa de confiabilidade do link (de 0% a 100%), sendo extremamente crítico.
        2. Indique se o link parece verdadeiro, falso ou suspeito.
        3. Explique os motivos da sua análise (uso de domínios parecidos com marcas famosas, URLs encurtadas, ausência de HTTPS, erros de digitação, pressa/urgência etc.).
        4. Diga, se possível, qual seria o domínio verdadeiro esperado para aquele serviço ou marca.
        5. Oriente a pessoa sobre como proceder ao receber links desse tipo (verificação no navegador, uso de antivírus, checagem no site oficial etc.).
        """,
        description="Agente verificador de links suspeitos"
    )
    return call_agent(agent, f"Analise este link: {link}")


def agente_verificador_mensagens_ligacoes(mensagem):
    agent = Agent(
        name="agente_verificador_mensagens_ligacoes",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em segurança digital com muita experiência em golpes, fraudes e engenharia social.
        Sua tarefa é analisar mensagens de texto e relatos de ligações (voz ou áudio) que podem ser suspeitas.

        1. Forneça uma taxa de confiabilidade da mensagem ou ligação (de 0% a 100%), sendo extremamente crítico.
        2. Avalie se o conteúdo parece verdadeiro, suspeito ou falso.
        3. Destaque sinais comuns de golpe ou tentativa de fraude (urgência, tom ameaçador, uso de instituições falsas, links perigosos, erros gramaticais etc.).
        4. Explique os indícios que tornam o conteúdo suspeito, e diga se o golpe é comum.
        5. Instrua a pessoa sobre como agir (não responder, não clicar, bloquear número, denunciar, etc.).
        """,
        description="Agente verificador de mensagens e ligações suspeitas"
    )
    return call_agent(agent, f"Analise esta mensagem: {mensagem}")


def agente_verificador_email(email):
    agent = Agent(
        name="agente_verificador_email",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em segurança digital com ampla experiência na análise de fraudes por e-mail.
        Sua tarefa é analisar cuidadosamente o endereço de e-mail e, se possível, o conteúdo associado a ele.

        1. Avalie a confiabilidade do e-mail (dê uma taxa de confiabilidade, de 0% a 100%, sendo extremamente crítico).
        2. Identifique se o e-mail é potencialmente fraudulento, falso, ou seguro.
        3. Destaque os elementos suspeitos no endereço de e-mail (nomes estranhos, domínios incomuns, uso de números ou letras trocadas, entre outros).
        4. Dê exemplos de como criminosos costumam forjar e-mails semelhantes aos reais.
        5. Oriente a pessoa sobre como agir diante de um possível golpe por e-mail.
        """,
        description="Agente verificador de emails suspeitos"
    )
    return call_agent(agent, f"Analise este email: {email}")


def agente_verificador_documentos(doc_info):
    agent = Agent(
        name="agente_verificador_documentos",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em segurança digital, com foco na identificação de documentos falsos, boletos fraudulentos e golpes envolvendo informações documentais.

        1. Analise com atenção as informações fornecidas sobre o documento ou boleto.
        2. Dê uma taxa de confiabilidade da autenticidade do documento(de 0% a 100%), sendo extremamente crítico.
        3. Identifique possíveis sinais de falsificação (como dados genéricos, erros de digitação, falta de consistência nas informações, CNPJs/CPFs inválidos, logos distorcidos, URLs falsas, entre outros).
        4. Explique por que o documento pode ser suspeito, dando exemplos reais de golpes comuns.
        5. Oriente claramente como a pessoa deve agir em caso de dúvida, como verificar o CNPJ em órgãos oficiais, usar aplicativos de leitura de boletos e evitar pagamentos antes da verificação.
        """,
        description="Agente verificador de documentos e boletos"
    )
    return call_agent(agent, f"Verifique este documento: {doc_info}")

# Menu principal
def menu():
    print("""
    🤖 Em que posso ajudar?

    🔗 1 - Verificar link
    📧 2 - Verificar email
    📢 3 - Analisar mensagem ou ligação
    📄 4 - Analisar documento(boleto, CNPJ, NFs, etc)
    🚨 5 - Fui vítima de um golpe digital
    📰 6 - Notícias sobre golpes digitais
    📚 7 - Boas práticas de segurança digital
    ❌ 0 - Sair
    """)
    return input("🤖 Digite o número da opção desejada: ").strip()


print("🔒 Bem-vindo(a) ao NãoCaIA 🔒\n\n🤖 Seu aliado na prevenção de fraudes e golpes digitais. 🤖\n\n\n🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸")


while True:
  try:
    opcao = menu()
    if opcao == "1":
        link = input("\n🔗 Digite o link para verificar: ")
        resposta = agente_verificador_link(link)
        display(to_markdown(resposta))

    elif opcao == "2":
        email = input("\n📧 Digite o email para verificar: ")
        resposta = agente_verificador_email(email)
        display(to_markdown(resposta))

    elif opcao == "3":
        mensagem = input("\n📢 Digite a mensagem ou descreva a ligação que voce recebeu: ")
        resposta = agente_verificador_mensagens_ligacoes(mensagem)
        display(to_markdown(resposta))

    elif opcao == "4":
        doc = input("\n📄 Descreva o documento suspeito para analise: ")
        resposta = agente_verificador_documentos(doc)
        display(to_markdown(resposta))

    elif opcao == "5":
        iniciar_chat_continuo(
            nome_agente="chat_ajuda_golpes",
            descricao="chat de ajuda para quem caiu em golpes digitais",
            instrucao="""
            Você é um assistente que conversa com o usuário e fornece orientações sobre como proceder caso ele tenha caído em um golpe digital.
            Use uma linguagem acessível. Aja como um advogado sério e profissional. Forneça um passo a passo, contatos para denúncia e mencione leis aplicáveis.
            """
        )
    elif opcao == "6":
        iniciar_chat_continuo(
            nome_agente="chat_noticias_golpes",
            descricao="chat de notícias sobre golpes digitais",
            instrucao="""
            Você é um assistente que conversa com o usuario e que fornece notícias recentes sobre golpes digitais, fraudes online e mudanças em leis relacionadas à cibersegurança.
            Informe com clareza, em linguagem simples para leigos, e com base nas informações atuais.
            """
        )

    elif opcao == "7":
        iniciar_chat_continuo(
            nome_agente="chat_boas_praticas",
            descricao="chat educativo sobre boas práticas de cibersegurança",
            instrucao="""
            Você é um educador em segurança digital. Responda dúvidas com dicas práticas sobre como proteger contas, reconhecer ameaças,
            configurar autenticação em dois fatores, criar senhas fortes, entre outros temas de cibersegurança, use uma linguagem para leigos no assunto.
            """
        )

    elif opcao == "0":
        print("\n🤖 Saindo... 👋")
        break

    else:
        print("\n🤖 Opção inválida. Tente novamente.")
  except Exception as e:
      print(f"\n❌ Ocorreu um erro: {e}")


🔒 Bem-vindo(a) ao NãoCaIA 🔒

🤖 Seu aliado na prevenção de fraudes e golpes digitais. 🤖


🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸🔸

    🤖 Em que posso ajudar?

    🔗 1 - Verificar link
    📧 2 - Verificar email
    📢 3 - Analisar mensagem ou ligação
    📄 4 - Analisar documento(boleto, CNPJ, NFs, etc)
    🚨 5 - Fui vítima de um golpe digital
    📰 6 - Notícias sobre golpes digitais
    📚 7 - Boas práticas de segurança digital
    ❌ 0 - Sair
    
🤖 Digite o número da opção desejada: 1

🔗 Digite o link para verificar: www.magalizeluiza.com.br


> ## Análise de Link: www.magalizeluiza.com.br
> 
> **Taxa de Confiabilidade:** 0%
> 
> **Status:** Falso
> 
> **Motivos:**
> 
> *   **Erro de Grafia:** O nome correto da empresa é "Magazine Luiza", e o domínio oficial é "magazineluiza.com.br". A inclusão de um "i" extra ("magalizeluiza") no link é uma tática comum de *phishing* para enganar usuários desatentos.
> *   **Domínio Similar:** Golpistas frequentemente usam domínios ligeiramente diferentes dos originais para se passar por empresas legítimas.
> 
> **Domínio Verdadeiro Esperado:**
> 
> *   www.magazineluiza.com.br
> 
> **Orientações:**
> 
> 1.  **Nunca clique no link.** Ele direciona para um site falso, possivelmente projetado para roubar suas informações pessoais (login, senha, dados bancários, etc.) ou infectar seu dispositivo com *malware*.
> 2.  **Verifique a URL:** Antes de clicar em qualquer link, passe o mouse sobre ele para visualizar o endereço completo na barra inferior do navegador. Observe atentamente se há erros de grafia, caracteres estranhos ou domínios suspeitos.
> 3.  **Acesse o site oficial diretamente:** Em vez de clicar no link, digite "magazineluiza.com.br" diretamente na barra de endereço do seu navegador.
> 4.  **Desconfie de promoções exageradas:** Se o link prometer ofertas muito vantajosas, desconfie. Golpistas usam promoções falsas para atrair vítimas.
> 5.  **Mantenha seu antivírus atualizado:** Um bom antivírus pode identificar e bloquear sites maliciosos.
> 6.  **Em caso de dúvida, entre em contato com a empresa:** Se você recebeu um e-mail ou mensagem supostamente da Magazine Luiza, mas está em dúvida sobre a autenticidade, entre em contato com o atendimento ao cliente da empresa pelos canais oficiais (site, telefone) para confirmar.
> 
> **Em resumo, o link "www.magalizeluiza.com.br" é uma tentativa de golpe. Mantenha-se vigilante e siga as orientações acima para proteger suas informações.**



    🤖 Em que posso ajudar?

    🔗 1 - Verificar link
    📧 2 - Verificar email
    📢 3 - Analisar mensagem ou ligação
    📄 4 - Analisar documento(boleto, CNPJ, NFs, etc)
    🚨 5 - Fui vítima de um golpe digital
    📰 6 - Notícias sobre golpes digitais
    📚 7 - Boas práticas de segurança digital
    ❌ 0 - Sair
    
